# Importing the Dependencies

In [8]:
import pandas as pd  
import numpy as np  
import os
import json
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

In [10]:
load_dotenv()

KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.3)